# Unüberwachtes Lernen

## Lernziele
* Dimensionsreduktion mittels Hauptkomponentenanalyse zur Reduktion der Anzahl von Attributen in einem Datensatz anwenden können.
* Den K Means Algorithmus zum Finden von Clustern in Daten anwenden können.
* Die Optimale Anzahl von Clustern mittels dem Silhouettenkoeffizienten finden können.

## Dimensionsreduktion


In der [Vorlesung](https://janalasser.at/lectures/MC_KI/VO3_3_merkmalskonstruktion_dimensionsreduktion/) haben wir Dimensionsreduktion als Werkzeug kennen gelernt, um dem [Fluch der Dimensionalität](https://janalasser.at/lectures/MC_KI/VO3_3_merkmalskonstruktion_dimensionsreduktion/#/2/2/3) zu entkommen. Das heißt, dass unser Datensatz *zu viele* verschiedene Attribute hat, um damit sinnvoll umgehen zu können. Insbesondere beim [unüberwachten Lernen](https://janalasser.at/lectures/MC_KI/VO3_1_unueberwachtes_lernen/) führt der Fluch der Dimensionalität dazu, dass die *Distanz* zwischen zwei Beobachtungen zunehmen an Bedeutung verliert, und Beobachtungen bzw. Gruppen von Beobachtungen (Cluster) schwerer voneinander zu unterscheiden sind.  


Im Folgenden wollen wir die Hauptkomponentenanalyse als Mittel zur Dimensionsduktion in Python umsetzen. Dazu verwenden wir wieder den Brustkrebs-Datensatz aus dem letzten Kurs.

In [ ]:
# lade Brustkrebsdaten
from sklearn import datasets
brustkrebs = datasets.load_breast_cancer(as_frame=True)

# teile Daten in Attribute und Zielwerte
x_brustkrebs = brustkrebs["data"]
y_brustkrebs = brustkrebs["target"]

x_brustkrebs.head()

Als erstes möchten wir uns ansehen, ob sich die Attribute von Beobachtungen mit Krebs systematisch von denen von Beobachtungen ohne Krebs unterscheiden. Dafür visualisieren wir die Daten mit Histogrammen und Punktwolken.

In [ ]:
import seaborn as sns
# da unser Datensatz 30 Attribute (Spalten) hat, beschränken wir uns zu
# Illustrationszwecken auf die ersten vier Attribute
interessante_spalten = ["mean radius", "mean texture", "mean perimeter", "mean area"]

# um die Daten einfach mit seaborn darstellen zu können, erstellen wir uns ein
# DataFrame, das die relevanten Attribute sowie die Zielwerte enthält
plot_daten_brustkrebs = x_brustkrebs[interessante_spalten].copy()
plot_daten_brustkrebs["target"] = y_brustkrebs.copy()

`seaborn` bietet die praktische Funktion `PairGrid()` (siehe [Dokumentation](https://seaborn.pydata.org/generated/seaborn.PairGrid.html)). Sie erzeugt ein Raster von Abbildungen mit jeweils so vielen Zeilen und Spalten wie Attribute im Datensatz enthalten sind.
* Auf der **Diagonalen** des Rasters wird je Attribut ein Histogramm der Attributswerte dargestellt.
* **Neben der Diagonalen** wird für jeweils zwei Attribute eine Punktwolke dargestellt.  

Insgesamt gibt es so für jede mögliche Kombination von zwei Attributen eine Punktwolke.  

In [ ]:
# wir Erzeugen ein Raster von Abbildungen
g = sns.PairGrid(plot_daten_brustkrebs, hue="target")
# wir legen fest, dass auf der Diagonalen Histogramme zu sehen sein sollen
g.map_diag(sns.histplot)
# wir legen fest, dass neben der Diagonalen Punktwolken zu sehen sein sollen
g.map_offdiag(sns.scatterplot, alpha=0.5)
# wir fügen manuell eine Legende hinzu, da diese nicht automatisch erzeugt wird
g.add_legend();

Wir sehen, dass die Punktwolken für Beobachtungen mit Krebs und solchen ohne Krebs sich durchaus unterscheiden - aber es gibt auch einen relativ größen Überlapp zwischen den Bereichen. Können wir die Punktwolken besser voneinander separieren?   

Dabei kann uns die Hauptkomponentenanalyse (principal component analysis, PCA) helfen. Wie unten illustriert, ist die Idee einer PCA, die einzelnen Dimensionen des Datensatz auf neue Dimensionen zu *projizieren*, die es einfacher machen, die Punktwolken voneinander zu unterscheiden.

In [ ]:
from IPython.display import Image
Image(url='https://upload.wikimedia.org/wikipedia/commons/9/9c/PCA_Projection_Illustration.gif')

In der [Vorlesung](https://janalasser.at/lectures/MC_KI/VO3_3_merkmalskonstruktion_dimensionsreduktion/#/2) haben wir die PCA schon als Werkzeug zur Dimensionsreduktion kennen gelernt. Auch für PCA bietet `scikit-learn` Funktionen, die teilweise sehr ähnlich zu denen funktionieren, die wir schon beim überwachten Lernen kennen gelernt haben.

In [ ]:
# wir importieren das PCA "Modell" aus scikit-learn
from sklearn.decomposition import PCA

# wir instantiieren ein leeres Modell und legen die Hyperparameter fest. Da wir
# nur die ersten beiden Hauptkomponenten (principal components) der Daten
# behalten wollen, legen wir n_components=2 fest
pca_brustkrebs = PCA(n_components=2)

# schlussendlich trainieren (fitten) wir das Modell auf den Brustkrebsdaten
pca_brustkrebs.fit(x_brustkrebs)

Im weiteren Vorgehen unterscheided sich die Anwendung von `pca()` aber von der des überwachten Lernens: wir wollen keine Werte für einzelne Beobachtungen vorhersagen sondern den gesamten Datensatz "transformieren". Deswegen verwenden wir die `transform()` Funktion des trainierten Modells.

In [ ]:
# wir transformieren die Daten indem wir sie auf die ersten beiden
# Hauptkomponenten projizieren. Übrig bleibt ein Datensatz, der nicht mehr 30
# Dimensionen hat sondern nur noch 2
x_brustkrebs_pca = pca_brustkrebs.transform(x_brustkrebs)
print("Originale Dimensionen des Datensatzes: {}".format(str(x_brustkrebs.shape)))
print("Reduzierte Dimensionen des Datensatzes: {}".format(str(x_brustkrebs_pca.shape)))

In [ ]:
# leider gehen beim Transformationsprozess die Spaltennamen verloren
x_brustkrebs_pca

In [ ]:
# wir erzeugen deshalb wieder ein pandas DataFrame aus den transformierten
# Daten und fügen manuell die Spaltennamen hinzu
import pandas as pd
x_brustkrebs_pca = pd.DataFrame(data=x_brustkrebs_pca, columns=["PC1", "PC2"])
x_brustkrebs_pca.head()

In [ ]:
# um die Daten zu visualisieren, fügen wir noch die Spalte "target" mit den
# Zielwerten aus den originalen (nicht transformierten) Daten hinzu
x_brustkrebs_pca["target"] = y_brustkrebs.copy()

# und stellen die auf die ersten beiden Hauptkomponenten projizierten Daten
# in einer Punktwolke dar
sns.scatterplot(data=x_brustkrebs_pca, x="PC1", y="PC2", hue="target", alpha=0.3)

Die Idee hinter der Hauptkomponentenanalyse ist, möglichst viel der im Datensatz enthaltenen *Varianz* in wenige Hauptkomponenten zu verschieben und damit effektiv die Dimensionen des Datensatzes zu reduzieren. Wie viel Varianz wird von den Hauptkomponenten jeweils erklärt? Diese Information ist in dem Attribut `explained_variance_ratio_` des "trainierten" PCA-Modells enthalten:

In [ ]:
pca_brustkrebs.explained_variance_ratio_

In [ ]:
# wir können das auch grafisch mit einem Balkendiagramm darstellen. Dafür
# basteln wir uns zuerst ein passendes DataFrame ...
erklaerte_varianz = pd.DataFrame()
erklaerte_varianz["komponente"] = range(1, len(pca_brustkrebs.explained_variance_ratio_) + 1)
erklaerte_varianz["erklaerte varianz"] = pca_brustkrebs.explained_variance_ratio_

# ... und verwenden dann die barplot() Funktion von seaborn
sns.barplot(data=erklaerte_varianz, x="komponente", y="erklaerte varianz")

Jede der Hauptkomponenten besteht aus einer Mischung (Linearkombination) der original im Datensatz enthaltenen Attribute. Wir können uns auch ansehen, welche Attribute für welche der Hauptkomponenten besonders wichtig sind. Diese Information ist im Attribut `components_` des "trainierten" PCA-Modells enthalten. Wobei wir hier über den Index auf die verschiedenen Hauptkomponenten zugreifen können. `components_[0]` z.B. enthält die Information darüber, wie wichtig die Attribute des Datensatzes jeweils für die erste Hauptkomponente sind, `components_[1]` für die zweite, usw.

In [ ]:
# auch die wichtigkeit der Attribute für die Hauptkomponenten können wir
# grafisch darstellen und basteln uns dafür ein pandas DataFrame.
wichtigkeit_attribute_brustkrebs = pd.DataFrame()
wichtigkeit_attribute_brustkrebs['attribut'] = x_brustkrebs.columns
wichtigkeit_attribute_brustkrebs['PC1'] = pca_brustkrebs.components_[0]
wichtigkeit_attribute_brustkrebs['PC2'] = pca_brustkrebs.components_[1]

In [ ]:
# Wichtigkeit der Attribute in der ersten Hauptkomponente
sns.barplot(data=wichtigkeit_attribute_brustkrebs, y='attribut', x='PC1')

In [ ]:
# Wichtigkeit der Attribute in der zweiten Hauptkomponente
sns.barplot(data=wichtigkeit_attribute_brustkrebs, y='attribut', x='PC2')

## Exkurs: Datenskalierung

Bis jetzt aben wir die Daten "as is" verwendet. Aber auch bei der PCA (und später beim unüberwachten Lernen bzw. Clustering) kann es nützlich sein, die Daten zuerst zu transformieren, um sie in einen ähnlichen Wertebereich zu bringen.  

Im Folgenden machen wir das einmal "zu Fuß" für das Attribut `mean radius` des Datensatzes:

In [ ]:
x_brustkrebs["mean radius"].head()

In [ ]:
# so sieht das Attribut aus, bevor wir es transformiert haben
sns.histplot(x_brustkrebs, x="mean radius")

In [ ]:
# als ersten Schritt "zentrieren" wir das Attribut, indem wir den Mittelwert
# aller Beobachtungen abziehen
x_brustkrebs["mean radius zentriert"] = x_brustkrebs["mean radius"] - x_brustkrebs["mean radius"].mean()
x_brustkrebs["mean radius zentriert"].head()

In [ ]:
# die Form der Verteilung ändert sich dadurch nicht, nur ihr Zentrum
sns.histplot(x_brustkrebs, x="mean radius zentriert")

In [ ]:
# als zweiten Schritt skalieren wir die Daten, indem wir sie durch die
# Standardabweichung aller Beobachtungen teilen
x_brustkrebs["mean radius skaliert"] = x_brustkrebs["mean radius zentriert"] / x_brustkrebs["mean radius"].std()
x_brustkrebs["mean radius skaliert"].head()

In [ ]:
# auch hier verändert sich die Form der Verteilung nicht, nur der von ihr
# abgedeckte Wertebereich
sns.histplot(x_brustkrebs, x="mean radius skaliert")

Wir müssen das Transformieren der Daten aber nicht "von Hand" erledigen - auch dafür liefert `scikit-learn` eingebaute Funktionalität. Die Transformationen die wir gerade durchgeführt haben (zentrieren, skalieren mit der Standardabweichung) werden vom `StandardScaler()` automatisch für alle Attribute (Spalten) des Datensatzes durchgeführt:

In [ ]:
# zuerst entfernen wir die beiden Spalten, die wir gerade manuell hinzugefügt
# haben wieder, da wir sie im späteren Verlauf nicht mehr brauchen
x_brustkrebs = x_brustkrebs.drop(columns=["mean radius zentriert", "mean radius skaliert"])

In [ ]:
# wir importieren den StandardScaler aus scikit-learn
from sklearn.preprocessing import StandardScaler

# wir erstellen einen "leeren" Skalierer ...
skalierer_brustkrebs = StandardScaler()

# ... und "trainieren" ihn auf dem Datensatz. Im Fall des StandardScaler
# passiert beim "Training" eigentlich nichts, außer dass der Funktion die Daten
# übergeben werden
skalierer_brustkrebs.fit(x_brustkrebs)

# wir skalieren den Daten mit der transform()-Funktion des Skalierers.
# die Funktion gibt uns die skalierten Daten zurück und wir speichern sie
# in einer neuen Variablen
x_brustkrebs_skaliert = skalierer_brustkrebs.transform(x_brustkrebs)

# leider gehen auch hier wieder die Spaltennamen verloren und wir fügen sie
# wieder manuell hinzu
x_brustkrebs_skaliert = pd.DataFrame(x_brustkrebs_skaliert, columns=x_brustkrebs.columns)
x_brustkrebs_skaliert.head()

Nun wollen wir erkunden, ob durch die Transformation der Daten die PCA besser in der Lage ist, die beiden Klassen ("krebs", "kein krebs") zu separieren.

In [ ]:
# wir instantiieren ein neues PCA modell und fitten es mit den skalierten
# Brustkrebsdaten
pca_brustkrebs_skaliert = PCA(n_components=2)
pca_brustkrebs_skaliert.fit(x_brustkrebs_skaliert)

# wir projizieren die skalierten Brustkrebsdaten auf die ersten beiden
# Hauptkomponenten
x_brustkrebs_pca_skaliert = pca_brustkrebs_skaliert.transform(x_brustkrebs_skaliert)

# auch hier sind wieder die Spaltennamen verlorengegangen und wir fügen sie
# hinzu, bevor wir die Daten visualisieren
x_brustkrebs_pca_skaliert = pd.DataFrame(data=x_brustkrebs_pca_skaliert, columns=["PC1", "PC2"])
x_brustkrebs_pca_skaliert["target"] = y_brustkrebs

# Visualisierung mit Punktwolke
sns.scatterplot(data=x_brustkrebs_pca_skaliert, x="PC1", y="PC2", hue="target", alpha=0.5)

In [ ]:
# die erklärte Varianz verteilt sich jetzt gleichmäßiger auf die beiden
# Hauptkomponenten
erklaerte_varianz_skaliert = pd.DataFrame()
erklaerte_varianz_skaliert["komponente"] = range(1, len(pca_brustkrebs_skaliert.explained_variance_ratio_) + 1)
erklaerte_varianz_skaliert["erklaerte varianz"] = pca_brustkrebs_skaliert.explained_variance_ratio_
sns.barplot(data=erklaerte_varianz_skaliert, x="komponente", y="erklaerte varianz")

In [ ]:
# und es fließen mehr Attribute in die Hauptkomponenten ein
wichtigkeit_attribute_brustkrebs_skaliert = pd.DataFrame()
wichtigkeit_attribute_brustkrebs_skaliert['attribut'] = x_brustkrebs_skaliert.columns
wichtigkeit_attribute_brustkrebs_skaliert['PC1'] = pca_brustkrebs_skaliert.components_[0]
wichtigkeit_attribute_brustkrebs_skaliert['PC2'] = pca_brustkrebs_skaliert.components_[1]

sns.barplot(data=wichtigkeit_attribute_brustkrebs_skaliert, y='attribut', x='PC1')

In [ ]:
sns.barplot(data=wichtigkeit_attribute_brustkrebs_skaliert, y='attribut', x='PC2')

## Übung 1

<font color='blue'>In der Code-Zelle unten wird ein Datensatz von Merkmalen von Blumen geladen, die verschiedenen Spezies angehören. Bei Interesse können Sie mehr Informationen zu dem Datensatz [hier](https://en.wikipedia.org/wiki/Iris_flower_data_set) nachlesen.
<ul class="outside">
<li><font color='blue'>Machen Sie sich mit dem Datensatz vertraut: wie viele Beobachtungen gibt es? Wie viele Merkmale und was bedeuten sie?</font></li>
<li><font color='blue'>Skalieren Sie die Daten indem Sie von jedem Attribut den Mittelwert abziehen und durch die Standardabweichung teilen. Hinweis: `StandardScaler()` von `scikit-learn` benutzen!</font></li>
<li><font color='blue'>Führen Sie eine Hauptkomponentenanalyse (PCA) mit `n_components=2` durch. Wie viel Varianz bilded die erste Hauptkomponente (PC 1) ab, wie viel die zweite (PC 2)?</font></li>
<li><font color='blue'>Welche Attribute des Datensatzes sind für die erste Hauptkomponente (PC 1) wichtig, welche für die zweite (PC 2)?</font></li>
</ul>
<font color='blue'></font>

In [ ]:
iris = datasets.load_iris(as_frame=True)
x_iris = iris["data"]
y_iris = iris["target"]

plot_daten_iris = x_iris.copy()
plot_daten_iris["target"] = y_iris.copy()

g = sns.PairGrid(plot_daten_iris, hue="target")
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot, alpha=0.5)
g.add_legend();

In [ ]:
# Skalierung: Ihr Code hier

In [ ]:
# PCA: ihr Code hier

In [ ]:
# Anteil der erklärten Varianz: Ihr Code hier

In [ ]:
# Wichtigkeit der Attribute: Ihr Code hier

## Clustering mit K-Means
Bis jetzt haben wir zu Illustrationszwecken immer noch die Klassenzugehörigkeit ("krebs", "kein krebs" bei den Brustkrebsdaten bzw. "setosa", "versicolor" und "virginica" bei den Blumen) verwendet. Wie gehen wir aber vor, wenn wir die Klassenzugehörigkeit nicht kennen sondern herausfinden wollen? Das ist der klassische Anwendungsfall für unüberwachtes Lernen bzw. [Clustering](https://janalasser.at/lectures/MC_KI/VO3_1_unueberwachtes_lernen/).  

Um das Konzept zu illustrieren, verwenden wir einen "echten" Datensatz von Merkmalen von Bauernhöfen, den wir auch in der [Vorlesung](https://janalasser.at/lectures/MC_KI/VO3_1_unueberwachtes_lernen/#/1/2) schon kurz besprochen haben. Zu diesem Datensatz gibt es auch ein Forschungsprojekt – bei Interesse können Sie Details in dieser [Publikation](https://www.nature.com/articles/s41598-021-00469-2) nachlesen. Unser Ziel wird es sein herauszufinden, ob die Bauernhöfe sich anhand ihrer Merkmale in unterschiedliche Gruppen aufteilen lassen.


In [ ]:
# wir laden den Datensatz mit Merkmalen von Bauernhöfen in ein pandas DataFrame
# er hat 160 Attribute aber keine Zielwerte
url = "https://drive.google.com/uc?id=1mNO7yf89ReYPvjJgfD3YdY5MdIVGvCtp"
farm = pd.read_csv(url)
farm.head()

In [ ]:
# wie für die Brustkrebsdaten skalieren wir die Daten ...
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(farm)
farm_skaliert = scaler.transform(farm)
farm_skaliert = pd.DataFrame(farm_skaliert, columns=farm.columns)

In [ ]:
# ... und führen eine PCA zur Dimensionsreduktion durch
from sklearn.decomposition import PCA
pca_farm = PCA(n_components=2)
pca_farm.fit(farm_skaliert)
farm_pca = pca_farm.transform(farm_skaliert)
farm_pca = pd.DataFrame(data=farm_pca, columns=["PC1", "PC2"])
sns.scatterplot(data=farm_pca, x="PC1", y="PC2", alpha=0.3)

Wir wollen nun, wie in der [Vorlesung](https://janalasser.at/lectures/MC_KI/VO3_1_unueberwachtes_lernen/#/0/3/8) beschrieben, eine Clusteranalyse durchführen, um verschiedene Cluster zu finden (die wir dann als "Klassen" interpretieren können). In der [Vorlesung](https://janalasser.at/lectures/MC_KI/VO3_4_ubueberwachtes_lernen_algorithmen/#/4) haben wir dafür den K-Means Algorithmus kennengelernt. Er findet Cluster, indem er ausgehend von einem bestehenden (zufälligen) Clustering am Anfang Beobachtungen in andere Cluster verschiebt, bis die Aufteilung der Beobachtungen in Cluster optimal ist.

In [ ]:
from IPython.display import Image
Image(url='https://upload.wikimedia.org/wikipedia/commons/e/ea/K-means_convergence.gif')

Auch für den K means Algorithmus bietet `scikit-learn` eine vorgefertigte Lösung die wir nach dem üblichen Schema verwenden können. Auch der K means Algorithmus hat Hyperparameter:
* `n_clusters`: wir müssen im Vorhinein festlegen, wie viele Cluster wir erwarten. Für die vorliegenden Daten zu Bauernhöfen wissen wir nicht wirklich, wie viele Cluster wir erwarten sollen. Wir können in diesem Fall nur raten, bzw. die Anzahl der Cluster systematisch durchprobieren.
* `random_state`: da das Clustering eine Zufallskomponente beinhaltet (welchem Cluster die Beobachtungen zu Beginn zugeordnet sind) ist es gut, den Zufall "festzuhalten" um Ergebnisse reproduzierbar zu machen.

In [ ]:
# importiere das KMeans Modell
from sklearn.cluster import KMeans

# instantiiere ein leeres Modell und lege die Hyperparameter fest.
# wir entscheiden uns im ersten Versuch für drei vorgegebene Cluster
kmeans_farm = KMeans(n_clusters=3, random_state=42)

# trainiere den Algorithmus auf den ersten beiden Hauptkomponenten der
# Brustkrebsdaten
kmeans_farm.fit(farm_pca)

# sage die Clusterzugehörigkeit der Beobachtungen voraus
cluster_farm = kmeans_farm.predict(farm_pca)

In [ ]:
cluster_farm

In [ ]:
# um die Clusterzugehörigkeit zu visualisieren, erstellen wir ein DataFrame
# mit den Hauptkomponenten und der vorhergesagten Clusterzugehörigkeit
plot_daten_farm = farm_pca.copy()
plot_daten_farm["cluster"] = cluster_farm

In [ ]:
sns.scatterplot(data=plot_daten_farm, x="PC1", y="PC2", hue="cluster", alpha=0.5, palette=sns.color_palette("tab10"))

Wie in der [Vorlesung](https://janalasser.at/lectures/MC_KI/VO3_4_ubueberwachtes_lernen_algorithmen/#/4/2/1) besprochen, hängt das Ergebnis des K Means Algorithmus vom gewählten Ausgangsclustering ab. Das Ausgangsclustering wird zufällig festgelegt – indem wir `random_state` auf eine Zahl setzen stellen wir sicher, dass es immer gleich gewählt wird. Was passiert, wenn wir einen anderen `random_state` setzen und damit ein anderes Ausgangsclustering wählen?

In [ ]:
# wir setzen den random_state auf 11 (war vorher 42) und lassen den Algorithmus
# nochmal laufen
kmeans_farm = KMeans(n_clusters=3, random_state=11)
kmeans_farm.fit(farm_pca)

# sage die Clusterzugehörigkeit der Beobachtungen voraus
cluster_farm_neuer_random_state = kmeans_farm.predict(farm_pca)

# Visualisierung der Daten mit vier Clustern
plot_daten_farm["cluster_4_rnd"] = cluster_farm_neuer_random_state
sns.scatterplot(data=plot_daten_farm, x="PC1", y="PC2", hue="cluster_4_rnd", alpha=0.5, palette=sns.color_palette("tab10"));

Was passiert, wenn wir die Anzahl der Cluster auf `n_clusters=4` setzen?

In [ ]:
# instantiiere ein leeres Modell und lege die Hyperparameter fest.
kmeans_farm = KMeans(n_clusters=4, random_state=11)

# trainiere den Algorithmus auf den ersten beiden Hauptkomponenten der
# Brustkrebsdaten
kmeans_farm.fit(farm_pca)

# sage die Clusterzugehörigkeit der Beobachtungen voraus
cluster_farm_4 = kmeans_farm.predict(farm_pca)

In [ ]:
cluster_farm_4

In [ ]:
# Visualisierung der Daten mit vier Clustern
plot_daten_farm["cluster_4"] = cluster_farm_4
sns.scatterplot(data=plot_daten_farm, x="PC1", y="PC2", hue="cluster_4", alpha=0.5, palette=sns.color_palette("tab10"));

## Die optimale Anzahl von Clustern finden

Wie können wir die beste Anzahl an Clustern herausfinden? Wir können versuchen, die Qualität des Clustering zu messen. Hierfür brauchen wir ein Maß bzw. eine Metrik für die Qualität des Clustering.  

In der [Vorlesung](https://janalasser.at/lectures/MC_KI/VO3_4_ubueberwachtes_lernen_algorithmen/#/4/3/3) haben wir den Silhouettenkoeffizient als möglichkeit zur Messung der Qualität eines Clusterings besprochen. Er misst, wie ähnlich eine Beobachtung anderen Beobachtungen im selben Cluster ist (Kohäsion) verglichen mit Beobachtungen in anderen Clustern (Separierung).

![silhouette score](https://drive.google.com/uc?id=17x-xSwIN5QUaB_IJ_Uw8IAgxAz7Pv0eF)

Auch für den Silhouettenkoeffizient gibt es in scikit-learn eine Funktion `silhouette_score()` im `metrics` Modul, die wir einfach verwenden können (siehe auch [Dokumentation](https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.silhouette_score.html)).  

Faustregel: Ein Clustering mit einem Silhouettenkoeffizienten von >= 0.7 ist ein "starkes" Clustering, ein Silhouettenkoeffizient von >= 0.5 ist ein "befriedigendes" Clustering, und ein Silhouettenkoeffizient von >= 0.25 ist ein "schwaches" Clustering. Werte darunter deuten darauf hin, dass es keine Cluster-Struktur in den Daten gibt. Insbesondere bei hochdimensionalen Daten ist bei der Interpretation aber Achtung geboten, da der [Fluch der Dimensionalität](https://janalasser.at/lectures/MC_KI/VO3_3_merkmalskonstruktion_dimensionsreduktion/#/2/2/3) dafür sorgt, dass die Werte des Silhouettenkoeffizienten insgesamt kleiner werden.

Alternativ können wir auch einen Clustering-Algorithmus wählen, der selbstständig die beste Anzahl von Clustern findet, wie z.B. DBSCAN (siehe Übung).

In [ ]:
# wir messen den Silhouettenkoeffizienten des Clusterings der Bauernhöfe
# mit 3 bzw. 4 vorgegebenen Clustern:
from sklearn.metrics import silhouette_score
sil_score_3 = silhouette_score(farm_pca, cluster_farm)
sil_score_4 = silhouette_score(farm_pca, cluster_farm_4)

print(f"Silhouette Score mit n_clusters=3: {sil_score_3}")
print(f"Silhouette Score mit n_clusters=4: {sil_score_4}")

## Übung 2
<ul class="outside">
<li><font color='blue'>Führen Sie das K Means Clustering für den "Iris"-Datensatz durch. Wählen Sie dafür einmal <tt>n_clusters=3</tt> und einmal <tt>n_clusters=2</tt>. Setzen Sie für beide Fälle den random state auf <tt>random_state=42</tt>. Speichern Sie die Ergebnisse jeweils in separaten Variablen.</font></li>
<li><font color='blue'>Visualisieren Sie das Clustering für beide Fälle mit Streudiagrammen wie schon für den Bauernhof-Datensatz.</font></li>
<li><font color='blue'>Berechnen Sie den Silhouettenkoeffizient für beide Fälle. Für welche Clusteranzahl ist er besser?</font></li>

## Hausaufgaben

Die Hausaufgaben für diesen Kursteil finden sich in [diesem Notebook](https://colab.research.google.com/drive/1sdDyGrwca9UIqw1QxwLF8Zk-_AI9NkeI?usp=sharing).

## Weiterführende Materialien
* **machine learning**: [Buch](https://www.amazon.de/Introduction-Machine-Learning-Python-Scientists/dp/1449369413?shipTo=AT&source=ps-sl-shoppingads-lpcontext&ref_=fplfs&psc=1&smid=A3JWKAKR8XB7XF&language=de_DE&gQT=1) Introduction to Machine Learning with Python: A Guide for Data Scientists mit umfassenden [Beispielen und Übungen](https://github.com/amueller/introduction_to_ml_with_python) in Python.

## Quelle und Lizenz

Das vorliegende Notebook wurde von Jana Lasser für den Kurs B "technische Aspekte" des Microcredentials "KI und Gesellschaft" der Universität Graz erstellt.

Das Notebook kann unter den Bedingungen der Lizenz [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0) verwendet, modifiziert und weiterverbreitet werden.
